In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import h5py

In [ ]:
# exp='tmolv2918'
exp='tmolw5618'
user='sample'
subdirectory='sample'

# Compare histograms

In [ ]:
plt.figure()
# for run in range(339,342):
for run in range(353,355):
    try:
        filepath = '/cds/data/psdm/tmo/%s/scratch/%s/slurm/%s/run-%d.h5' % (exp, user, subdirectory, run)
        with h5py.File(filepath, 'r') as data:
            print(data.keys())
            evrs = np.copy(data['evrs']).astype(bool)
            print(np.nanmax(data['photonEnergy']))
    #         goodShots = (~evrs[:,161])
            goodShots = np.ones(evrs.shape[0]).astype(bool)
    #         print(evrs.shape,goodShots.shape,goodShots.dtype)
            ampl = np.array(data['MBa'])[goodShots,:][:]
            times = np.array(data['MBt'])[goodShots,:][:]
            ampl0 = ampl[~np.isnan(ampl)]
            times0 = times[~np.isnan(times)]

            hist, edges= np.histogram(times0, bins=2000, range=(0,2))
    #         plt.hist(times, bins=200, range=(0,2),);
            plt.plot(edges[1:], hist, label=str(run))
    except OSError:
        continue
plt.xlim([.25,2])
plt.legend()

In [ ]:
filepath = '/cds/data/psdm/tmo/tmolw5618/scratch/sample/slurm/c01188-run-353.h5'
data = h5py.File(filepath,'r')
# !ls $filepath
def printH5keys(filepath):
    with h5py.File(filepath, 'r') as data:
        return list(data.keys())
printH5keys(filepath)

# Verify uniqueness of data

In [ ]:
evrs = np.copy(data['evrs']).astype(bool)
print(evrs.shape)

In [ ]:
mbWaveform = data['mb-waveform-downsample']#[evrs[:,68],:]
# itofWaveform = data['iToF_wf'][evrs[:,68],:]

In [ ]:
print(mbWaveform.shape)
print(np.unique(np.sum(mbWaveform,-1)).size)

In [ ]:
mbWaveform[0,:]-mbWaveform[-1,:]

# Verify timestamps look ok
This was problem in lv29

In [ ]:
timestamps = np.array(data['timestamp'])

Verify unique timestamps

In [ ]:
print(timestamps.shape, np.unique(timestamps).size)

Verify frequency after time-sorting

In [ ]:
print('Expected run duration is ... %f minutes' % (timestamps.size*8*1e-3/60.) )

In [ ]:
sortedTimestamps = np.sort(timestamps)
timeRange = sortedTimestamps[-1]-sortedTimestamps[0]

In [ ]:
print('Timestamps indicate a run duration of ... %f minutes' % (timeRange*1e-9/60.))

# Epics analysis example

In [ ]:
data['epics'].keys()

In [ ]:
vitaraDelay = np.array(data['epics']['las_fs14_target_time_dial'])
plt.plot(vitaraDelay[:500])
plt.xlabel('time idx')
plt.ylabel('vitara delay (ns)')

# MB analysis example

In [ ]:
loadMax = 1000
evrs = np.array( data['evrs'][:loadMax,] ).astype(int)
mbWaveform = np.array( data['mb-waveform'][:loadMax,] )
print(evrs.shape, mbWaveform.shape)
t = np.array( data['mb-time'][:loadMax,] )[0,:]

plt.plot(t, mbWaveform.mean(0), 'k', label='Jet on');
plt.title("MB traces");
plt.legend()
plt.xlabel("ToF / us")

# Test correlations for off-by-one errors
Repeat example for VLS sum vs. pulse energy
Should look correlated if all is well

In [ ]:
pulseEnergy = data['pulseEnergy']
electronCounts = np.nansum(~np.isnan(data['MBt']),-1)
plt.scatter(pulseEnergy,electronCounts)